In [ ]:
import csv

with open('static/knowledge_sanphams.csv', mode='r', newline='', encoding='utf-8') as f:
    knowledge_sanphams = [e for e in csv.DictReader(f)]

In [ ]:
gr_jsonapi = {
  "Khách hàng": {
    "Tên": "CÔNG TY CỔ PHẦN PC1 MIỀN NAM",
    "Địa chỉ": "-1"
  },
  "Danh sách vật tư": [
    {
      "Vật tư": "Thép F08 - AI",
      "Xuất xứ": "-1",
      "Khối lượng - Số lượng": {
        "Giá trị": 2338.82,
        "Đơn vị": "kg"
      },
      "Ghi chú vật tư": "-1",
      "Mã nội bộ": "-1"
    },
    {
      "Vật tư": "Thép F10 - AIII",
      "Xuất xứ": "hoà phát",
      "Khối lượng - Số lượng": {
        "Giá trị": 1144.77,
        "Đơn vị": "kg"
      },
      "Ghi chú vật tư": "-1",
      "Mã nội bộ": "-1"
    },
    {
      "Vật tư": "Thép F14 - AIII",
      "Xuất xứ": "pomina",
      "Khối lượng - Số lượng": {
        "Giá trị": 9079.69,
        "Đơn vị": "kg"
      },
      "Ghi chú vật tư": "-1",
      "Mã nội bộ": "-1"
    },
  ],
  "Ghi chú chung": "Thời gian đề nghị cấp: ngày 06/02/2025\nĐịa điểm tiếp nhận: Tại vị trí móng 39 thuộc Tam Phước - Biên Hòa - Đồng Nai. Công ty Minh Phương Mr\nĐịnh SDT: +84 977 408 985"
}

for i, e in enumerate(gr_jsonapi['Danh sách vật tư']):
    print(e['Vật tư'])
    print(e['Xuất xứ'])
    gr_jsonapi['Danh sách vật tư'][i]['Mã nội bộ'] = "..."

Thép F08 - AI
-1
Thép F10 - AIII
hoà phát
Thép F14 - AIII
pomina
